In [1]:
# Importando Librerías
import re
import json
import time
import pprint
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from selenium import webdriver
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
import warnings
warnings.filterwarnings("ignore")
pd.set_option('display.max_rows', None)


In [2]:
# with open('data/parts/jojo_parts.json', 'w', encoding='utf-8') as file:
#     json.dump(jojo_data, file, indent=4, sort_keys=False)

In [3]:
# Creando WebDriver para la busqueda con Chrome
driver = webdriver.Chrome(ChromeDriverManager().install())


In [4]:
#Ingresando a página para obtener información de las partes de Jojo's
page_url = f'https://jojowiki.com/Category:JoJo%27s_Bizarre_Adventure_Parts'
driver.get(page_url)

## Hallando URL/Imagen cada parte 

In [5]:
# Encontrando partes
jojo_images = driver.find_elements(
    By.XPATH, '//*[@id="mw-content-text"]/div[2]/div[1]/div/div[2]/div/div[1]/a/img')
jojo_url = driver.find_elements(
    By.XPATH, '//*[@id="mw-content-text"]/div[2]/div[1]/div/div[2]/div/div[2]/a'
)

jojo_parts = {}
for image, url in zip(jojo_images, jojo_url):

    title = url.text
    url = url.get_attribute('href')

    image = image.get_attribute('src')

    image = image.replace('/thumb','').split('/')
    image = '/'.join(image[:-1])
    
    jojo_parts[title] = {"url": url}
    jojo_parts[title]["image"] = image
    

# with open('data/parts/jojo_parts.json', 'w', encoding='utf-8') as file:
#     json.dump(jojo_parts, file, indent=4, sort_keys=False)

jojo_parts

{'Phantom Blood': {'url': 'https://jojowiki.com/Phantom_Blood',
  'image': 'https://static.jojowiki.com/images/b/b1/latest/20191015213555/Volume_1.jpg'},
 'Battle Tendency': {'url': 'https://jojowiki.com/Battle_Tendency',
  'image': 'https://static.jojowiki.com/images/3/34/latest/20191015213237/Volume_11.jpg'},
 'Stardust Crusaders': {'url': 'https://jojowiki.com/Stardust_Crusaders',
  'image': 'https://static.jojowiki.com/images/0/0a/latest/20191015212952/Volume_17.jpg'},
 'Diamond is Unbreakable': {'url': 'https://jojowiki.com/Diamond_is_Unbreakable',
  'image': 'https://static.jojowiki.com/images/b/ba/latest/20191015213904/Volume_36.jpg'},
 'Vento Aureo': {'url': 'https://jojowiki.com/Vento_Aureo',
  'image': 'https://static.jojowiki.com/images/4/40/latest/20191015215618/Volume_49.jpg'},
 'Stone Ocean': {'url': 'https://jojowiki.com/Stone_Ocean',
  'image': 'https://static.jojowiki.com/images/a/a6/latest/20191015214320/Volume_80.jpg'},
 'Steel Ball Run': {'url': 'https://jojowiki.co

## Hallando Info por Parte

In [6]:
jojo_data = jojo_parts.copy()

In [7]:
partes = list(jojo_data.keys())
partes

['Phantom Blood',
 'Battle Tendency',
 'Stardust Crusaders',
 'Diamond is Unbreakable',
 'Vento Aureo',
 'Stone Ocean',
 'Steel Ball Run',
 'JoJolion']

## Characters

In [ ]:
xpath = {'Phantom Blood': '//*[@id="mw-content-text"]/div[2]/div[14]/div/div[2]/div/div[1]/a',
         'Battle Tendency': '//*[@id="mw-content-text"]/div[2]/div[15]/div/div[2]/div/div[1]/a',
         'Stardust Crusaders':'//*[@id="mw-content-text"]/div[2]/div[31]/div/div[2]/div/div[1]/p/span[1]/a',
         'Diamond is Unbreakable':'//*[@id="mw-content-text"]/div[2]/div[22]/div/div[2]/div/div[1]/p/span[1]/a',
         'Vento Aureo':''}


In [ ]:
for part in jojo_data.keys():
    driver.get(jojo_data[part]["url"])

    fields = driver.find_elements(
        By.XPATH, '//*[@id="mw-content-text"]/div[2]/aside/div/h3')

    datum = driver.find_elements(
        By.XPATH, '//*[@id="mw-content-text"]/div/aside/div/div')

    for field, data in zip(fields, datum):
        resume = ' '.join([driver.find_element(
            By.XPATH, f'//*[@id="mw-content-text"]/div[2]/p[{i}]').text for i in range(3, 6)])

        resume = re.sub("\[.*?\]", "", resume)
        jojo_data[part]["resume"] = resume
        jojo_data[part][field.text.lower()] = data.text

    print(part.upper(), '\t', jojo_data[part]["url"])
    temp = {}
    for character in characters:
        name = character.get_attribute('title')
        url = character.get_attribute('href')
        print(name, url)
        
        
    #     temp[name] = {"url":url}
    # jojo_data[part]["characters"] = temp
    
    print('-'*45)
    break


In [ ]:
temp

In [ ]:
jojo_data

In [ ]:
with open('test.json', 'w', encoding='utf-8') as file:
    json.dump(jojo_data, file)